# Source localization pipeline - Stage2
- coregistration
- source localization
- report generation
 
 


#### Author
Rudi Kreidenhuber <Rudi.Kreidenhuber@gmail.com>
#### License
BSD (3-clause)

# Coregistration

In [ ]:
from src import SubjectDropDowner
import os
import glob
import mne

# configuration
input_dir = "..\\..\\MEG_playground\\processed"
bids_root = "..\\..\\MEG_playground\\results"
n_jobs = openmp = 3
extras_directory = os.path.join(input_dir, "extras")
FS_SUBJECTS_DIR = os.path.abspath("..\\..\\fs_subjects_dir")
spacing = "ico4"

print("Configuration:")
for dir in [input_dir, bids_root, extras_directory, FS_SUBJECTS_DIR]:
    print(f"{dir}")


def get_subject_list(input_dir = input_dir) -> list:
    fiflist = [os.path.basename(f) for f in glob.glob(os.path.join(input_dir, "*.fif"))]
    fiflist = [f.split("_")[0] for f in fiflist]
    return fiflist

subjectlist = get_subject_list()

dl = SubjectDropDowner.SubjectDropDowner(subjectlist)
drop_menu = dl.create_subject_dropdown_widget()
print(f"\n\nChoose patient/ subject to process further:")
drop_menu

In [ ]:
subject = dl.get_subject_dropdown().value

if not subject.startswith("sub-"):
    subject = "sub-" + subject
    
print(f"Chosen Subject: {subject}")



In [ ]:
from src import Folderer, Anatomist

# create output folders
dfc = Folderer.DerivativesFoldersCreator(BIDS_root=bids_root, 
                                            extras_directory=extras_directory, 
                                            subject=subject)
dfc.make_derivatives_folders()

# copy freesurfer files to local subjects_dir
try:
    segmentation = os.path.join(os.path.abspath(FS_SUBJECTS_DIR), subject)
    target = os.path.join(dfc.fanat, subject)
    if not os.path.isdir(target):
        os.mkdir(target)
    dfc._recursive_overwrite(segmentation, target)
except Exception as e:
    print(f"Couldn't copy freesurfer segmentation\n--> {e}.")
    
# create source models
sourcerer = Anatomist.SourceModeler(subjects_dir=dfc.fanat, subject=subject, spacing=spacing, n_jobs=n_jobs)
sourcerer.calculate_source_models()


transfile = os.path.join(dfc.ftrans, subject + "-trans.fif")
print("Starting coregistration...")
if os.path.isfile(transfile):
    print(f"Skipping coregistration, because a transfile ({transfile}) already exists")
else:
    print(f"\n\n\n--> Transfile should be called: {transfile}\n\n\n")
    try:
        mne.gui.coregistration(subject=subject, subjects_dir=dfc.fanat, inst=bids_root, advanced_rendering=False) # BIDS: inst=raw.filenames[0])
    except:
        print("failed with bids_derivatives folder")
        filename = subject + "_prepped.fif"
        rawfile = os.path.join(input_dir, "processed", filename)
        print(f"Coregistration with BIDS-file failed, Rawfile used was: {rawfile}")
        mne.gui.coregistration(subject=subject, subjects_dir=dfc.fanat, inst=rawfile, advanced_rendering=False)